# Face Mask Detection with YOLO (Real-time)

This notebook builds a face mask detection system optimized for real-time speed and accuracy using the Ultralytics YOLO family. It covers setup, data preparation (YOLO format), training, evaluation, inference, webcam demo, performance tuning, and model export.

Folder location: `notebooks/face-mask-detection-yolo.ipynb`

Classes: `no_mask`, `mask`

## 1) Set Up Environment and GPU

Detect CUDA, select device (cpu/cuda), set deterministic flags, and print versions for reproducibility.

In [ ]:
# Environment and GPU setup
import os, sys, platform, random
from pathlib import Path

# Torch may be installed later; keep this cell tolerant if torch isn't present yet
try:
    import torch
    TORCH_AVAILABLE = True
except Exception:
    TORCH_AVAILABLE = False

# Device selection
CUDA_AVAILABLE = TORCH_AVAILABLE and torch.cuda.is_available()
DEVICE = 'cuda' if CUDA_AVAILABLE else 'cpu'

# Determinism and seeds
SEED = 42
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
if TORCH_AVAILABLE:
    torch.manual_seed(SEED)
    if CUDA_AVAILABLE:
        torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Print versions for reproducibility
print({
    'python': sys.version.split()[0],
    'platform': platform.platform(),
    'device': DEVICE,
    'torch': torch.__version__ if TORCH_AVAILABLE else 'not-installed',
})

# Optional: show GPUs
if CUDA_AVAILABLE:
    print('CUDA GPU count:', torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f'- [{i}]', torch.cuda.get_device_name(i))

## 2) Install Dependencies

Installs required libraries. GPU is recommended but not required. If you want ONNX GPU runtime, include onnxruntime-gpu.

In [ ]:
# Install packages if missing
import importlib, sys, subprocess

def pip_install(pkg):
    try:
        importlib.import_module(pkg.split('==')[0].split('[')[0])
        return
    except Exception:
        pass
    print(f'Installing {pkg} ...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

# Core deps
for pkg in ['ultralytics', 'opencv-python', 'onnxruntime-gpu; platform_system=="Windows"']:
    try:
        if ';' in pkg:  # handle environment markers simply
            base, marker = pkg.split(';', 1)
            # basic check: only install on Windows for the example marker
            if 'Windows' in marker and platform.system() == 'Windows':
                pip_install(base)
        else:
            pip_install(pkg)
    except Exception as e:
        print('Note: optional package failed or skipped:', pkg, e)

# Verify imports and versions
import cv2
from ultralytics import YOLO
print('cv2:', cv2.__version__)
print('ultralytics:', YOLO.__module__.split('.')[0])

# Torch and CUDA checks
try:
    import torch
    print('torch:', torch.__version__, '| cuda available:', torch.cuda.is_available())
except Exception as e:
    print('torch not installed or failed to import:', e)

## 3) Define Project Paths

Define ROOT, data_dir, runs_dir, and weights paths relative to the notebook.

In [4]:
# Paths relative to this notebook
from pathlib import Path
ROOT = Path.cwd().resolve()
DATA_DIR = ROOT.parent / 'ComputerVision' / 'FaceMaskDetection_YOLO' / 'data'
RUNS_DIR = ROOT.parent / 'ComputerVision' / 'FaceMaskDetection_YOLO' / 'runs'
WEIGHTS_DIR = RUNS_DIR / 'weights'

for p in [DATA_DIR, RUNS_DIR, WEIGHTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print('ROOT:', ROOT)
print('DATA_DIR:', DATA_DIR)
print('RUNS_DIR:', RUNS_DIR)
print('WEIGHTS_DIR:', WEIGHTS_DIR)

ROOT: E:\VSCODE\Data-Science-Monorepo\ComputerVision\notes
DATA_DIR: E:\VSCODE\Data-Science-Monorepo\ComputerVision\ComputerVision\FaceMaskDetection_YOLO\data
RUNS_DIR: E:\VSCODE\Data-Science-Monorepo\ComputerVision\ComputerVision\FaceMaskDetection_YOLO\runs
WEIGHTS_DIR: E:\VSCODE\Data-Science-Monorepo\ComputerVision\ComputerVision\FaceMaskDetection_YOLO\runs\weights


## 4) Download or Prepare Dataset (YOLO format)

Expected structure under `data/face-mask/`:
- train/images, train/labels
- val/images, val/labels
- test/images, test/labels

Each label file uses YOLO txt format per image: `class cx cy w h` normalized to [0,1].

Pick one option below: unzip a local archive or ensure the folders already exist.

#link: https://www.kaggle.com/datasets/aditya276/face-mask-dataset-yolo-format

In [ ]:
# Create expected directories if missing and optionally unzip a local archive
#link: https://www.kaggle.com/datasets/aditya276/face-mask-dataset-yolo-format
from pathlib import Path
import shutil, zipfile

FM_ROOT = DATA_DIR / 'dataset'
for split in ['train', 'val', 'test']:
    (FM_ROOT / split / 'images').mkdir(parents=True, exist_ok=True)
    (FM_ROOT / split / 'labels').mkdir(parents=True, exist_ok=True)

print('Dataset root:', FM_ROOT)

# Option A: Unzip an existing archive placed in DATA_DIR (e.g., face-mask-yolo.zip)
ARCHIVE = DATA_DIR / 'face_mask.zip'
if ARCHIVE.exists():
    print('Found archive, unzipping:', ARCHIVE)
    with zipfile.ZipFile(ARCHIVE, 'r') as zf:
        zf.extractall(DATA_DIR)
    print('Unzip done. Ensure extracted structure matches expected YOLO layout.')
else:
    print('No archive found at', ARCHIVE)

# Minimal validation: number of images vs labels in each split
for split in ['train', 'val', 'test']:
    imgs = sorted((FM_ROOT / split / 'images').glob('*.*'))
    labels = sorted((FM_ROOT / split / 'labels').glob('*.txt'))
    print(split, 'images:', len(imgs), '| labels:', len(labels))

## 5) Create data.yaml

Create a YOLO data config with paths and class names ['mask', 'no_mask', 'mask_incorrect'].

In [5]:
import yaml

data_yaml = {
    'path': str(FM_ROOT),
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': 2,
    'names': ['no_mask', 'mask']
}

DATA_YAML_PATH = DATA_DIR / 'data.yaml'
with open(DATA_YAML_PATH, 'w') as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)

print('Wrote', DATA_YAML_PATH)
print(Path(DATA_YAML_PATH).read_text())

NameError: name 'FM_ROOT' is not defined

## 6) Visualize Training Samples

Randomly preview labeled images to verify dataset quality.

In [ ]:
import random, os
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

names = data_yaml['names']  # ['no_mask', 'mask']
colors = [tuple(np.random.randint(0,255,3).tolist()) for _ in names]

img_paths = sorted((FM_ROOT / 'train' / 'images').glob('*.*'))
random.shuffle(img_paths)

def draw_yolo_boxes(img_path):
    img = cv2.imread(str(img_path))
    if img is None:
        return None
    h, w = img.shape[:2]
    label_path = (img_path.parent.parent / 'labels' / (img_path.stem + '.txt'))
    if label_path.exists():
        for line in Path(label_path).read_text().strip().splitlines():
            cls, cx, cy, bw, bh = map(float, line.split())
            cls = int(cls)
            x1 = int((cx - bw/2) * w)
            y1 = int((cy - bh/2) * h)
            x2 = int((cx + bw/2) * w)
            y2 = int((cy + bh/2) * h)
            color = colors[cls % len(colors)]
            cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
            if 0 <= cls < len(names):
                cv2.putText(img, names[cls], (x1, max(0,y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            else:
                cv2.putText(img, str(cls), (x1, max(0,y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

cols = 3
rows = 2
plt.figure(figsize=(12,8))
for i, p in enumerate(img_paths[:cols*rows]):
    img = draw_yolo_boxes(p)
    if img is None:
        continue
    plt.subplot(rows, cols, i+1)
    plt.imshow(img)
    plt.axis('off')
plt.tight_layout()
plt.show()

## 7) Train YOLO Model

We use a small YOLO model (yolov8n) for real-time performance. Adjust epochs and batch size as needed.

In [ ]:
# GPU diagnostics: Torch/Torchvision/CUDA & NMS availability
try:
    import torch, torchvision
    print({
        'torch': torch.__version__,
        'torch_cuda_available': torch.cuda.is_available(),
        'torch_cuda_version': getattr(torch.version, 'cuda', 'none'),
        'torchvision': getattr(torchvision, '__version__', 'unknown'),
    })
    if torch.cuda.is_available():
        print('GPU count:', torch.cuda.device_count())
        for i in range(torch.cuda.device_count()):
            print(f'- [{i}]', torch.cuda.get_device_name(i))
    try:
        from torchvision.ops import nms
        dev = 'cuda' if torch.cuda.is_available() else 'cpu'
        boxes = torch.tensor([[0,0,10,10],[1,1,9,9]], dtype=torch.float32, device=dev)
        scores = torch.tensor([0.9,0.8], dtype=torch.float32, device=dev)
        try:
            idx = nms(boxes, scores, 0.5)
            loc = 'CUDA' if dev == 'cuda' else 'CPU'
            print(f'torchvision.ops.nms OK on {loc}. kept indices:', idx.tolist())
        except NotImplementedError as e:
            print('NotImplementedError from torchvision::nms -> missing CUDA ops; training will fall back to CPU with AMP disabled unless you install a CUDA-enabled torchvision matching torch/CUDA.')
    except Exception as e:
        print('torchvision.ops.nms import/run failed:', e)
    # Show what the notebook thinks the device is
    try:
        print('Notebook DEVICE variable:', DEVICE)
    except NameError:
        pass
except Exception as e:
    print('Diagnostics failed:', e)

In [ ]:
from ultralytics import YOLO

# Initialize model (nano for speed; switch to 'yolov8s.pt' for better accuracy)
base_weights = 'yolov8n.pt'
model = YOLO(base_weights)

EPOCHS = int(os.getenv('EPOCHS', 55))
BATCH = int(os.getenv('BATCH', 10))
IMGSZ = int(os.getenv('IMGSZ', 640))
WORKERS = int(os.getenv('WORKERS', 6))

# Print torch/torchvision versions to help debug CUDA NMS issues
try:
    import torch, torchvision
    print('torch:', torch.__version__, '| cuda available:', torch.cuda.is_available())
    print('torchvision:', getattr(torchvision, '__version__', 'unknown'))
except Exception as _e:
    print('Version probe failed:', _e)

# Train with graceful fallback if CUDA NMS is unavailable in torchvision
try:
    results = model.train(
        data=str(DATA_YAML_PATH),
        imgsz=IMGSZ,
        epochs=EPOCHS,
        batch=BATCH,
        device=0 if DEVICE=='cuda' else 'cpu',
        workers=WORKERS,
        project=str(RUNS_DIR),
        name='train/mask-yolo',
        patience=20,
        # leave amp=True by default; Ultralytics will run an AMP check
    )
except NotImplementedError as e:
    msg = str(e)
    if 'torchvision::nms' in msg and DEVICE == 'cuda':
        print('\n[WARN] CUDA NMS from torchvision is not available for your build.\n'
              'Falling back to CPU training with AMP disabled.\n'
              'To restore GPU training, install a torchvision build that matches your torch/CUDA.\n')
        results = model.train(
            data=str(DATA_YAML_PATH),
            imgsz=IMGSZ,
            epochs=EPOCHS,
            batch=BATCH,
            device='cpu',
            workers=max(0, min(WORKERS, 2)),  # be conservative on Windows
            project=str(RUNS_DIR),
            name='train/mask-yolo',
            patience=20,
            amp=False,
            val=False,  # skip val to avoid NMS during training; you can run val later explicitly
        )
    else:
        raise

print('Training complete. Best weights should be in runs/train/mask-yolo/weights/best.pt')

## 8) Evaluate Model Performance

Compute mAP, precision/recall, and plot confusion matrix and PR curves.

In [2]:
# Load best weights and evaluate
best_weights = RUNS_DIR / 'train' / 'mask-yolo' / 'weights' / 'best.pt'
assert best_weights.exists(), f"Best weights not found: {best_weights}"
model = YOLO(str(best_weights))

val_results = model.val(data=str(DATA_YAML_PATH), imgsz=IMGSZ, device=0 if DEVICE=='cuda' else 'cpu', project=str(RUNS_DIR), name='val')
print(val_results.results_dict)

# The Ultralytics API saves confusion_matrix.png and PR curves under runs/val
print('Val artifacts saved to:', RUNS_DIR / 'val')

NameError: name 'RUNS_DIR' is not defined

## 9) Run Inference on Sample Images

Run predictions on the test split and save annotated images.

In [ ]:
from ultralytics import YOLO
from pathlib import Path

model = YOLO(str(best_weights))

# Ensure class name mapping is correct for display
yaml_names = data_yaml['names']
try:
    # model.names can be dict or list; normalize and override when lengths match
    if hasattr(model, 'names'):
        if isinstance(model.names, dict):
            current = [model.names[i] for i in sorted(model.names.keys())]
        else:
            current = list(model.names)
        if len(current) == len(yaml_names):
            model.names = {i: n for i, n in enumerate(yaml_names)}
except Exception:
    pass

TEST_IMAGES_DIR = FM_ROOT / 'test' / 'images'
PRED_OUT = RUNS_DIR / 'predict'
PRED_OUT.mkdir(parents=True, exist_ok=True)

res = model.predict(source=str(TEST_IMAGES_DIR), imgsz=IMGSZ, conf=0.25, iou=0.5, save=True, save_txt=False, project=str(RUNS_DIR), name='predict')
print('Predictions saved to:', RUNS_DIR / 'predict')

# Print a short summary for the first few images
names_map = model.names if isinstance(model.names, dict) else {i:n for i,n in enumerate(yaml_names)}
for r in res[:5]:
    boxes = r.boxes
    if boxes is None:
        continue
    print(Path(r.path).name, 'detections:')
    for b in boxes:
        cls = int(b.cls.item())
        conf = float(b.conf.item())
        xyxy = b.xyxy[0].tolist()
        label = names_map.get(cls, str(cls))
        print('  -', label, f'{conf:.2f}', [round(v,1) for v in xyxy])

## 10) Real-Time Webcam Inference

Press 'q' to quit the stream. If you have multiple cameras, try index 1 or 2. On Windows PowerShell, ensure camera permissions are allowed.

In [6]:
# Self-contained real-time webcam inference (run this cell alone)
import os
import time
from pathlib import Path

import cv2
import numpy as np
import torch
from ultralytics import YOLO

# 1) Resolve project paths relative to this notebook
ROOT = Path.cwd().resolve()  # .../ComputerVision/notes
BASE_DIR = ROOT.parent / 'FaceMaskDetection_YOLO'
DATA_YAML_PATH = BASE_DIR / 'data' / 'data.yaml'
RUNS_DIR = BASE_DIR / 'runs'

# 2) Find best weights automatically (or use BEST_WEIGHTS env var)
def find_best_weights() -> Path | None:
    cand_env = os.getenv('BEST_WEIGHTS')
    if cand_env:
        p = Path(cand_env)
        if p.exists():
            return p
        print(f"[WARN] BEST_WEIGHTS set but not found: {p}")
    # common default location from training section
    default = RUNS_DIR / 'train' / 'mask-yolo' / 'weights' / 'best.pt'
    if default.exists():
        return default
    # fallback: pick most recent best.pt anywhere under runs
    if RUNS_DIR.exists():
        bests = list(RUNS_DIR.rglob('best.pt'))
        if bests:
            bests.sort(key=lambda p: p.stat().st_mtime, reverse=True)
            return bests[0]
    return None

best_path = find_best_weights()
assert best_path is not None and best_path.exists(), ("No best.pt found. Set BEST_WEIGHTS env var to a valid path or run training.\n"
                                                     f"Searched under: {RUNS_DIR}")
print('Using weights:', best_path)

# 3) Load model
model = YOLO(str(best_path))

# 4) Determine class names for overlay
names: list[str] | None = None
# Try to read from data.yaml first (keeps your chosen order; e.g., ['no_mask', 'mask'])
try:
    import yaml  # lazy import
    if DATA_YAML_PATH.exists():
        with open(DATA_YAML_PATH, 'r', encoding='utf-8') as f:
            cfg = yaml.safe_load(f) or {}
        yaml_names = cfg.get('names')
        if isinstance(yaml_names, (list, tuple)) and len(yaml_names) > 0:
            names = list(yaml_names)
            print('Class names from data.yaml:', names)
except Exception as _:
    pass

# Fallback to model.names if needed
try:
    model_names = model.names if hasattr(model, 'names') else None
    if names is None or (isinstance(model_names, (dict, list)) and len(model_names) == len(names) == 0):
        if isinstance(model_names, dict) and model_names:
            names = [model_names[i] for i in sorted(model_names.keys())]
        elif isinstance(model_names, list) and model_names:
            names = list(model_names)
except Exception:
    pass

# Final guard: default names if still missing
if not names:
    names = ['class_0', 'class_1']
    print('[WARN] Falling back to default class names:', names)

# Optional: enforce display mapping to data.yaml order when lengths match
try:
    if hasattr(model, 'names') and len(names) == (len(model.names) if not isinstance(model.names, dict) else len(model.names.keys())):
        model.names = {i: n for i, n in enumerate(names)}
except Exception:
    pass

# 5) Device/precision
CUDA = torch.cuda.is_available()
DEVICE_ARG = 0 if CUDA else 'cpu'
HALF = True if CUDA else False  # Ultralytics handles casting; we toggle via flag only

# 6) Colors for classes
rng = np.random.default_rng(123)
colors = [tuple(int(c) for c in rng.integers(0, 255, 3)) for _ in names]

# 7) Open webcam (set CAM_INDEX env var to change camera)
cam_index = int(os.getenv('CAM_INDEX', '0'))
cap = cv2.VideoCapture(cam_index)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

if not cap.isOpened():
    raise RuntimeError(f'Webcam index {cam_index} not accessible. Try a different CAM_INDEX (e.g., 1 or 2).')

prev = time.time()
print('Starting webcam. Press q to quit...')
while True:
    ok, frame = cap.read()
    if not ok:
        print('Failed to grab frame')
        break

    img = frame.copy()

    # Inference (retry in float32 if half precision dtype mismatch occurs)
    try:
        results = model.predict(source=img, imgsz=640, conf=0.35, iou=0.5, device=DEVICE_ARG, half=HALF, verbose=False)
    except RuntimeError as e:
        if 'dtype' in str(e).lower():
            print('[WARN] dtype mismatch with half precision; retrying with half=False')
            HALF = False
            results = model.predict(source=img, imgsz=640, conf=0.35, iou=0.5, device=DEVICE_ARG, half=False, verbose=False)
        else:
            raise

    r = results[0]
    if r.boxes is not None:
        for b in r.boxes:
            cls = int(b.cls.item())
            conf = float(b.conf.item())
            x1, y1, x2, y2 = map(int, b.xyxy[0].tolist())
            color = colors[cls % len(colors)]
            label = names[cls] if 0 <= cls < len(names) else str(cls)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f"{label} {conf:.2f}", (x1, max(0, y1 - 6)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    now = time.time()
    fps = 1.0 / max(1e-6, (now - prev))
    prev = now
    cv2.putText(img, f'FPS: {fps:.1f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    cv2.imshow('Face Mask Detection - YOLO', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Using weights: E:\VSCODE\Data-Science-Monorepo\ComputerVision\FaceMaskDetection_YOLO\runs\train\mask-yolo\weights\best.pt
Class names from data.yaml: ['no_mask', 'mask']
Starting webcam. Press q to quit...
Starting webcam. Press q to quit...


## 11) Performance Tuning for Real-Time

Quickly benchmark FPS with different image sizes and half precision settings.

In [ ]:
import time, numpy as np, cv2
from ultralytics import YOLO
import torch

model = YOLO(str(best_weights))
CUDA = torch.cuda.is_available()
if CUDA:
    model.to('cuda')


def benchmark(imgsz_list=(320, 480, 640), half_list=(False, True), warmup=5, iters=30):
    results = []
    dummy = np.random.randint(0,255,(720,1280,3), dtype=np.uint8)
    for imgsz in imgsz_list:
        for half in half_list:
            if half and not CUDA:
                continue
            # warmup
            for _ in range(warmup):
                model.predict(source=dummy, imgsz=imgsz, device=0 if CUDA else 'cpu', half=half, verbose=False)
            t0 = time.time()
            for _ in range(iters):
                model.predict(source=dummy, imgsz=imgsz, device=0 if CUDA else 'cpu', half=half, verbose=False)
            dt = time.time() - t0
            fps = iters / dt
            results.append({'imgsz': imgsz, 'half': half, 'fps': round(fps,1)})
    return results

bench = benchmark()
for r in bench:
    print(r)

print('Tip: yolov8n is faster, yolov8s is more accurate; choose based on your target FPS and accuracy.')

## 12) Export Model (ONNX) and Verify

Export the trained model to ONNX and verify with ONNX Runtime.

In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2, os

model = YOLO(str(best_weights))
export_path = model.export(format='onnx', dynamic=True, opset=12)
print('Exported ONNX path:', export_path)

# Verify with ONNX Runtime
try:
    import onnxruntime as ort
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if ort.get_device() == 'GPU' else ['CPUExecutionProvider']
    sess = ort.InferenceSession(export_path, providers=providers)

    # Prepare one dummy frame
    dummy = np.random.randint(0,255,(640,640,3), dtype=np.uint8)
    img = cv2.cvtColor(dummy, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (640,640))
    img = img.transpose(2,0,1)[None].astype(np.float32) / 255.0

    inputs = {sess.get_inputs()[0].name: img}
    outputs = sess.run(None, inputs)
    print('ONNX forward ok. Output shapes:', [np.array(o).shape for o in outputs])
except Exception as e:
    print('ONNX Runtime verification skipped or failed:', e)

## 13) Save and Load Trained Weights

Document the weights path and reload to test a quick prediction.

In [ ]:
from ultralytics import YOLO

print('Best weights:', best_weights)
model = YOLO(str(best_weights))

# Run a quick single-image test if any test image exists
sample_imgs = sorted((FM_ROOT / 'test' / 'images').glob('*.*'))
if sample_imgs:
    res = model.predict(source=str(sample_imgs[0]), imgsz=IMGSZ, conf=0.25, save=True, project=str(RUNS_DIR), name='quickcheck')
    print('Quick check saved to:', RUNS_DIR / 'quickcheck')
else:
    print('No test images found for quick check.')

## 14) Minimal Unit Tests (Postprocessing and I/O)

Lightweight checks to catch obvious issues without heavy dependencies.

In [ ]:
# Simple checks
assert len(data_yaml['names']) == 2
assert data_yaml['names'][0] == 'no_mask' and data_yaml['names'][1] == 'mask'
assert set(Path(str(FM_ROOT)).glob('**/images')).__class__  # existence of paths check

# Color mapping reproducibility
import numpy as np
rng = np.random.default_rng(123)
colors = [tuple(rng.integers(0,255,3).tolist()) for _ in data_yaml['names']]
assert len(colors) == 2

# Webcam availability check (non-fatal)
import cv2
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Webcam not accessible — skipping live capture test.')
else:
    ok, frame = cap.read()
    print('Webcam frame shape:' if ok else 'Failed to read a frame')
    if ok:
        print(frame.shape)
cap.release()

## 15) Notebook File Path

This notebook is saved at: `notebooks/face-mask-detection-yolo.ipynb`

Training artifacts and exports are saved under: `Computer Vision/FaceMaskDetection_YOLO/runs/`

In [ ]:
# Dataset summary and quick visualization grid for YOLO labels
# Safe to run multiple times; it reads paths from data.yaml
from pathlib import Path
import random
import math
import yaml
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# 1) Load dataset config
DATA_YAML = Path(r"E:\VSCODE\Data-Science-Monorepo\ComputerVision\FaceMaskDetection_YOLO\data\data.yaml")
with open(DATA_YAML, 'r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

root = Path(cfg.get('path') or DATA_YAML.parent)
# cfg['train'] / cfg['val'] / cfg['test'] are like 'train/images'

def split_dirs(key: str):
    rel = Path(cfg.get(key, f"{key}/images"))
    img_dir = root / rel
    lbl_dir = img_dir.parent / 'labels'
    return img_dir, lbl_dir

splits = ['train', 'val', 'test']

# 2) Print counts per split
summary = []
for s in splits:
    img_dir, lbl_dir = split_dirs(s)
    imgs = list(img_dir.glob('*.*')) if img_dir.exists() else []
    # Only count common image extensions
    imgs = [p for p in imgs if p.suffix.lower() in {'.jpg', '.jpeg', '.png', '.bmp'}]
    lbls = list(lbl_dir.glob('*.txt')) if lbl_dir.exists() else []
    summary.append((s, len(imgs), len(lbls), img_dir, lbl_dir))

print("Dataset root:", root)
for s, ni, nl, img_dir, lbl_dir in summary:
    print(f"{s:>5}: images={ni:5d} | labels={nl:5d} | {img_dir.relative_to(root)}  /  {lbl_dir.relative_to(root)}")

# 3) Visualize a grid of labeled images for a chosen split
#    If you want another split, set SPLIT='val' or 'test'.
SPLIT = 'train'
N = 12  # number of images to show
cols = 4
rows = math.ceil(N / cols)

img_dir, lbl_dir = split_dirs(SPLIT)
assert img_dir.exists(), f"Image dir not found: {img_dir}"
assert lbl_dir.exists(), f"Label dir not found: {lbl_dir}"

names = cfg.get('names') or []

# Helper to read yolo txt labels
# Each line: class x_center y_center width height (normalized to [0,1])
def read_yolo_labels(label_path, img_w, img_h):
    boxes = []
    if not label_path.exists():
        return boxes
    with open(label_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:  # some datasets might have more (xywh + conf), keep first 5
                parts = parts[:5]
            try:
                cls, xc, yc, w, h = parts
                cls = int(float(cls))
                xc, yc, w, h = map(float, (xc, yc, w, h))
            except Exception:
                continue
            # convert to pixel xyxy
            bw = w * img_w
            bh = h * img_h
            cx = xc * img_w
            cy = yc * img_h
            x1 = max(0, cx - bw / 2)
            y1 = max(0, cy - bh / 2)
            boxes.append((cls, x1, y1, bw, bh))  # store as (cls, x1,y1,w,h)
    return boxes

# Get candidate images
images = [p for p in img_dir.glob('*.*') if p.suffix.lower() in {'.jpg', '.jpeg', '.png', '.bmp'}]
if not images:
    print(f"No images found in {img_dir}")
else:
    random.shuffle(images)
    sel = images[:N]

    fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 3.5*rows))
    if not isinstance(axes, (list, tuple)):
        axes = axes.reshape(rows, cols)

    for ax, img_path in zip(axes.flat, sel):
        # Load image via matplotlib (no OpenCV dependency)
        img = plt.imread(str(img_path))
        if img.dtype.kind == 'f':  # some readers return float [0,1]
            img_h, img_w = img.shape[:2]
        else:
            img_h, img_w = img.shape[:2]

        label_path = lbl_dir / (img_path.stem + '.txt')
        boxes = read_yolo_labels(label_path, img_w, img_h)

        ax.imshow(img)
        ax.set_title(img_path.name, fontsize=8)
        ax.axis('off')

        for cls, x1, y1, bw, bh in boxes:
            rect = patches.Rectangle((x1, y1), bw, bh, linewidth=1.5, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
            if 0 <= cls < len(names):
                ax.text(x1, y1 - 2, names[cls], color='yellow', fontsize=7,
                        bbox=dict(facecolor='black', alpha=0.4, pad=1))

    # Hide any unused axes
    for i in range(len(sel), rows*cols):
        axes.flat[i].axis('off')

    plt.tight_layout()
    plt.show()